In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import h5py
import torch
from mpl_toolkits.axes_grid1 import ImageGrid
from scipy.interpolate import griddata
import random
import copy
import cv2 as cv

In [2]:
def filter_images(images, threshold):
    filtered_images = []
    indexnew = []
    count = 0
    for image in images:
        count = count+1
        y = np.argwhere(np.isnan(image))
        num_nan_pixels = y.shape[0]
        total_pixels = image.shape[0] * image.shape[1]
        percent_nan = num_nan_pixels / total_pixels
        if percent_nan <= threshold:
            filtered_images.append(image)
            indexnew.append(count)
    return filtered_images, indexnew

In [3]:
def interpolate_missing_values(image):
    # Create a binary mask where True indicates missing values
    mask = np.isnan(image)

    # Get the coordinates of valid (non-missing) pixels
    coords = np.argwhere(~mask)

    # Get the pixel values at the valid coordinates
    values = image[~mask]

    # Generate a grid of coordinates for the missing pixels
    missing_coords = np.argwhere(mask)

    # Interpolate the missing pixel values using griddata
    interpolated_values = griddata(coords, values, missing_coords, method='linear')

    # Replace the missing pixel values with the interpolated values
    interpolated_image = np.copy(image)
    interpolated_image[mask] = interpolated_values

    return interpolated_image

In [4]:
def image_impute(images,indexs):
    imputed_images = []
    for idx,image in enumerate(images):
        print(idx)
        if idx in indexs:
            im = interpolate_missing_values(image)
            imputed_images.append(im)
        else:
            imputed_images.append(image)
    return imputed_images

PMW data

In [3]:
IRIndexs = np.load('irIndexs.npy')
PMWIndexs = np.load('pmwIndexs.npy')

In [4]:
finalIndexs = list(set(IRIndexs).intersection(set(PMWIndexs)))

In [5]:
keepimagep = []
originalIndexp = []
for i in range(len(PMWIndexs)):
    if PMWIndexs[i] in IRIndexs:
        keepimagep.append(i)
        originalIndexp.append(PMWIndexs[i])

In [6]:
keepimager = []
originalIndexr = []
for i in range(len(IRIndexs)):
    if IRIndexs[i] in PMWIndexs:
        keepimager.append(i)
        originalIndexr.append(IRIndexs[i])

In [7]:
len(originalIndexp)

69456

In [8]:
len(originalIndexr)

69456

In [10]:
count = 0
for i in range(len(originalIndexp)):
    if originalIndexp[i] == finalIndexs[i]:
        count = count+1

if count == len(originalIndexp):
    print('SAME')

SAME


In [49]:
PMWData = np.load('PMWData_Imputed.npy')

In [51]:
PMWData = PMWData[keepimagep,:,:]

In [52]:
PMWData.shape

(69456, 224, 224)

In [53]:
np.save('./FinalDataset/PMWData.npy',PMWData)

In [54]:
del PMWData

In [11]:
dataInfo = '/home/desktop/Desktop/22104412_Docs/CVDL/Tropical Cyclone Intensity Estimation/TCIR/TCIR-ATLN_EPAC_WPAC.h5'
Info1 = pd.read_hdf(dataInfo, key="info",mode = 'r')

In [13]:
dataInfo = '/home/desktop/Desktop/22104412_Docs/CVDL/Tropical Cyclone Intensity Estimation/TCIR/TCIR-CPAC_IO_SH.h5'
Info2 = pd.read_hdf(dataInfo, key = "info", mode = 'r')

In [14]:
Info = pd.concat((Info1,Info2))

In [15]:
vmax = np.array(Info['Vmax'])

In [17]:
vmax = vmax[finalIndexs]
vmax.shape

(69456,)

In [18]:
np.save('VmaxTrain.npy',vmax)

In [31]:
TestData = np.load('./FinalDataset/TestData.npy')

In [32]:
TestData.shape

(4561, 201, 201, 2)

In [26]:
indexNan

array([], dtype=int64)

In [27]:
Filtered, indexfiltered = filter_images(dataIR, 0.1)

NameError: name 'filter_images' is not defined

In [28]:
indexfiltered = np.array(indexfiltered)
indexfiltered = indexfiltered-1
indexfiltered = indexfiltered.tolist()
len(indexfiltered)

NameError: name 'indexfiltered' is not defined

In [ ]:
fig = plt.figure(figsize=(20., 20.))
grid = ImageGrid(fig, 111, 
                 nrows_ncols=(10,10),  # creates 2x2 grid of axes
                 axes_pad=0,  # pad between axes
                )

for ax, im in zip(grid, Filtered[60200:60300]):
    ax.imshow(im, cmap = 'gray')

# plt.savefig('examples_NaN_threshold_0.1.png')
plt.show()

Lets impute this. Then we will have 69493 images without Nan.
Then we will do the same for the PMW data. Then we will take the indices common for both and concatenate both

In [ ]:
z = np.where(np.isnan(Filtered))
z = (np.unique(z[0])).tolist()

In [ ]:
len(z)

In [ ]:
Filtered = image_impute(Filtered,z)

In [ ]:
for i in t:
    Image = Filtered[i]
    nan_ind = np.where(np.isnan(Image))
    Image[np.isnan(Image)] = np.mean(Image[~nan_ind[0],~nan_ind[1]])
    Filtered[i] = Image

In [ ]:
t = np.where(np.isnan(Filtered))

In [ ]:
t = (np.unique(t[0])).tolist()

In [ ]:
t

In [ ]:
np.save('pmwIndexs.npy', indexfiltered)

In [ ]:
#Filtered is my IR data array
np.save('PMWData_Imputed.npy',Filtered)